In [ ]:
import math
import numpy as np
import random
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
# Creating hrrs
def hrr(length, normalized=False):
    if normalized:
        x = np.random.uniform(-np.pi,np.pi,int((length-1)/2))
        if length % 2:
            x = np.real(np.fft.ifft(np.concatenate([np.ones(1), np.exp(1j*x), np.exp(-1j*x[::-1])])))
        else:
            x = np.real(np.fft.ifft(np.concatenate([np.ones(1), np.exp(1j*x), np.ones(1), np.exp(-1j*x[::-1])])))
    else:
        x = np.random.normal(0.0, 1.0/np.sqrt(length), length)
    return x

In [ ]:
# Convolve two hrrs
def convolve(x, y):
    return np.real(np.fft.ifft(np.fft.fft(x)*np.fft.fft(y)))

# Pre convolve all hrrs to save time
def preconvolve():
    preconvolved_matrix = np.zeros([possible_wm.size, size_of_maze, n])
    for x in range(len(goal_states)):
        for y in range(size_of_maze):
            preconvolved_matrix[x][y] = convolve(possible_wm[x], state_hrrs[y])
    return preconvolved_matrix

In [ ]:
# Returns reward
def get_reward(state, goal_state):
    return reward_good if state == goal_state else reward_bad

In [ ]:
# Policy for choosing left or right
def policy_mov(state, wm, rand_on):
    
    # Calculates what the left and right positions on the maze are
    if (state == 0):
        state_left = size_of_maze - 1
        state_right = state + 1
    elif (state == size_of_maze - 1):
        state_left = state - 1
        state_right = 0
    else:
        state_left = state - 1
        state_right = state + 1
    
    # Value of each move
    value_left_state_wm = np.dot(weights, preconvolved_matrix[wm][state_left]) + bias
    value_right_state_wm = np.dot(weights, preconvolved_matrix[wm][state_right]) + bias
    
    # Random move
    if((np.random.random() < e_soft) and (rand_on == 1)):
        return np.random.choice([state_left, state_right])
    
    # Finds best move
    max_value = max(value_left_state_wm, value_right_state_wm)
    
    # Returns best move
    if(max_value == value_left_state_wm):
        return state_left
    elif(max_value == value_right_state_wm):
        return state_right
    
# Switching wm depending on td error
def policy_switch(wm):
    return (wm + 1)%2

In [ ]:
# Size of hrrs
n = 1024

# Maze creation
size_of_maze = 4

# Setting goals
goal_states = random.sample(range(0, size_of_maze), 2)

# Create all state hrrs
state_hrrs = np.zeros([size_of_maze, n])
for x in range(size_of_maze):
    state_hrrs[x] = hrr(n, True)
    
# Create all wm hrrs
possible_wm = np.zeros([len(goal_states), n])
for x in range(len(goal_states)):
    possible_wm[x] = hrr(n, True)

# Reward hrr
reward_tkn = hrr(n, True)

# Pre convolve hrrs to save time
preconvolved_matrix = preconvolve()

# Weights and bias of neural network
weights = hrr(n, True)
bias = 0

# Reward for learning
reward_bad = 0
reward_good = 1

# Discounted future rewards
discount = 0.5

# Learning rate
alpha = 0.01

# Exploration on or off
exp_on = 1

# Exploration rate
e_soft = 0.01

# Treshold to switch wm
threshold = -0.50

# Number of training cycles
episodes = 100000

# Steps to try finding the goal before quiting
steps_till_quit = 100

# For switching tasks
count = 0

# Frequency to print information
print_freq = 1000

# wm is set to first
wm = 0

In [ ]:
for x in range(episodes):
    
    # Current state
    current = random.randint(0, size_of_maze - 1)
    
    # Switching tasks
    count += 1
    goal_state = goal_states[0] if count == 1 or count == 21 else goal_states[1] if count == 11 else goal_state
    # Resetting count
    if count == 21: count = 1

    for y in range (steps_till_quit):
        # Gets reward for current state
        r = get_reward(current, goal_state)
        
        # Current state value
        current_state = preconvolved_matrix[wm][current]
        current_value = np.dot(weights, current_state) + bias
        
        # Store info about previous state
        previous = current
        previous_state = current_state
        previous_wm = wm
        previous_value = current_value
        
        # Goal reached
        if (current == goal_state):
            # Get temporal difference error and update weights of neural network
            td_error = r - previous_value
            weights += (alpha * td_error * convolve(reward_tkn, previous_state))
            break
        
        # What state to move into
        move = policy_mov(current, wm, exp_on)
        
        # Make the move
        current = move
        current_state = preconvolved_matrix[wm][current]
        current_value = np.dot(weights, current_state) + bias

        # Weight update for goal not found
        td_error = (r + discount*current_value) - previous_value
        if td_error < threshold:
            wm = policy_switch(wm)
            break
        weights += (alpha * td_error * previous_state)
    if (x+1)%print_freq == 0:
        print("Episode", x+1, "done")

In [ ]:
position = np.arange(size_of_maze)
value = np.zeros(size_of_maze)
for z in range (size_of_maze):
    if z == goal_states[0]:
        value[z] = np.dot(weights, (convolve(reward_tkn, preconvolved_matrix[0][z])))
    else:
        value[z] = np.dot(weights, preconvolved_matrix[0][z])
plt.plot(position, value)
print(np.amax(value))

In [ ]:
position = np.arange(size_of_maze)
value = np.zeros(size_of_maze)
for z in range (size_of_maze):
    if z == goal_states[1]:
        value[z] = np.dot(weights, (convolve(reward_tkn, preconvolved_matrix[1][z])))
    else:
        value[z] = np.dot(weights, preconvolved_matrix[1][z])
plt.plot(position, value) 
print(np.amax(value))